In [0]:
from pyspark.sql import SparkSession
from urllib.request import urlretrieve
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col, count, isnan, when
from pyspark.sql.types import IntegerType

In [0]:
project_path   = "/ames/data/"
database_name = "ames"
table_name = "ames_data"

In [0]:
dbutils.fs.mkdirs(project_path)

Out[140]: True

In [0]:
display(dbutils.fs.ls("/"))

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/ames/,ames/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/dbacademy/,dbacademy/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
spark.sql(f"USE {database_name}");

In [0]:
url = "http://jse.amstat.org/v19n3/decock/AmesHousing.txt"
file = "AmesHousing.txt"
urlretrieve(url, file)
driver_path = "file:/databricks/driver/" + file
file_path = project_path + file
dbutils.fs.mv(driver_path, file_path)
df = spark.read.format("csv").option("delimiter", "\t").option("header", True).option("inferSchema", True).csv(file_path)

In [0]:
df.select( "PID", "MS SubClass", "MS Zoning", "Lot Frontage", "Lot Area").show(5)

+---------+-----------+---------+------------+--------+
 PID|MS SubClass|MS Zoning|Lot Frontage|Lot Area|
+---------+-----------+---------+------------+--------+
526301100| 20| RL| 141| 31770|
526350040| 20| RH| 80| 11622|
526351010| 20| RL| 81| 14267|
526353030| 20| RL| 93| 11160|
527105010| 60| RL| 74| 13830|
+---------+-----------+---------+------------+--------+
only showing top 5 rows

In [0]:
df = df.select([col(c).alias(c.replace(" ", "_")) for c in df.columns])

In [0]:
sql("SET spark.databricks.delta.formatCheck.enabled=false")
df.write.format("delta").mode("overwrite").save(project_path)

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS {table_name}
""")

spark.sql(f"""
CREATE TABLE {table_name}
USING DELTA
LOCATION "{project_path}"
""")

Out[147]: DataFrame[]

In [0]:
ames_df = spark.sql(f"""
SELECT * FROM {table_name}
""")
display(ames_df.take(5))

Order,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Year_Built,Year_Remod/Add,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Heating,Heating_QC,Central_Air,Electrical,1st_Flr_SF,2nd_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Functional,Fireplaces,Fireplace_Qu,Garage_Type,Garage_Yr_Blt,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,3Ssn_Porch,Screen_Porch,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Yr_Sold,Sale_Type,Sale_Condition,SalePrice
1,526301100,20,RL,141,31770,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112,TA,TA,CBlock,TA,Gd,Gd,BLQ,639,Unf,0,441,1080,GasA,Fa,Y,SBrkr,1656,0,0,1656,1,0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960,Fin,2,528,TA,TA,P,210,62,0,0,0,0,NA,NA,NA,0,5,2010,WD,Normal,215000
2,526350040,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,TA,5,Typ,0,NA,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal,105000
3,526351010,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Gd,6,Typ,0,NA,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal,172000
4,526353030,20,RL,93,11160,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0,Gd,TA,CBlock,TA,TA,No,ALQ,1065,Unf,0,1045,2110,GasA,Ex,Y,SBrkr,2110,0,0,2110,1,0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968,Fin,2,522,TA,TA,Y,0,0,0,0,0,0,NA,NA,NA,0,4,2010,WD,Normal,244000
5,527105010,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,PConc,Gd,TA,No,GLQ,791,Unf,0,137,928,GasA,Gd,Y,SBrkr,928,701,0,1629,0,0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997,Fin,2,482,TA,TA,Y,212,34,0,0,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal,189900


In [0]:
ames_df.columns

Out[149]: ['Order',
 'PID',
 'MS_SubClass',
 'MS_Zoning',
 'Lot_Frontage',
 'Lot_Area',
 'Street',
 'Alley',
 'Lot_Shape',
 'Land_Contour',
 'Utilities',
 'Lot_Config',
 'Land_Slope',
 'Neighborhood',
 'Condition_1',
 'Condition_2',
 'Bldg_Type',
 'House_Style',
 'Overall_Qual',
 'Overall_Cond',
 'Year_Built',
 'Year_Remod/Add',
 'Roof_Style',
 'Roof_Matl',
 'Exterior_1st',
 'Exterior_2nd',
 'Mas_Vnr_Type',
 'Mas_Vnr_Area',
 'Exter_Qual',
 'Exter_Cond',
 'Foundation',
 'Bsmt_Qual',
 'Bsmt_Cond',
 'Bsmt_Exposure',
 'BsmtFin_Type_1',
 'BsmtFin_SF_1',
 'BsmtFin_Type_2',
 'BsmtFin_SF_2',
 'Bsmt_Unf_SF',
 'Total_Bsmt_SF',
 'Heating',
 'Heating_QC',
 'Central_Air',
 'Electrical',
 '1st_Flr_SF',
 '2nd_Flr_SF',
 'Low_Qual_Fin_SF',
 'Gr_Liv_Area',
 'Bsmt_Full_Bath',
 'Bsmt_Half_Bath',
 'Full_Bath',
 'Half_Bath',
 'Bedroom_AbvGr',
 'Kitchen_AbvGr',
 'Kitchen_Qual',
 'TotRms_AbvGrd',
 'Functional',
 'Fireplaces',
 'Fireplace_Qu',
 'Garage_Type',
 'Garage_Yr_Blt',
 'Garage_Finish',
 'Garage_Cars',
 'Garage_Area',
 'Garage_Qual',
 'Garage_Cond',
 'Paved_Drive',
 'Wood_Deck_SF',
 'Open_Porch_SF',
 'Enclosed_Porch',
 '3Ssn_Porch',
 'Screen_Porch',
 'Pool_Area',
 'Pool_QC',
 'Fence',
 'Misc_Feature',
 'Misc_Val',
 'Mo_Sold',
 'Yr_Sold',
 'Sale_Type',
 'Sale_Condition',
 'SalePrice']

In [0]:
ames_df.printSchema()

root
-- Order: integer (nullable = true)
-- PID: integer (nullable = true)
-- MS_SubClass: integer (nullable = true)
-- MS_Zoning: string (nullable = true)
-- Lot_Frontage: integer (nullable = true)
-- Lot_Area: integer (nullable = true)
-- Street: string (nullable = true)
-- Alley: string (nullable = true)
-- Lot_Shape: string (nullable = true)
-- Land_Contour: string (nullable = true)
-- Utilities: string (nullable = true)
-- Lot_Config: string (nullable = true)
-- Land_Slope: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Condition_1: string (nullable = true)
-- Condition_2: string (nullable = true)
-- Bldg_Type: string (nullable = true)
-- House_Style: string (nullable = true)
-- Overall_Qual: integer (nullable = true)
-- Overall_Cond: integer (nullable = true)
-- Year_Built: integer (nullable = true)
-- Year_Remod/Add: integer (nullable = true)
-- Roof_Style: string (nullable = true)
-- Roof_Matl: string (nullable = true)
-- Exterior_1st: string (nullable = true)
-- Exterior_2nd: string (nullable = true)
-- Mas_Vnr_Type: string (nullable = true)
-- Mas_Vnr_Area: integer (nullable = true)
-- Exter_Qual: string (nullable = true)
-- Exter_Cond: string (nullable = true)
-- Foundation: string (nullable = true)
-- Bsmt_Qual: string (nullable = true)
-- Bsmt_Cond: string (nullable = true)
-- Bsmt_Exposure: string (nullable = true)
-- BsmtFin_Type_1: string (nullable = true)
-- BsmtFin_SF_1: integer (nullable = true)
-- BsmtFin_Type_2: string (nullable = true)
-- BsmtFin_SF_2: integer (nullable = true)
-- Bsmt_Unf_SF: integer (nullable = true)
-- Total_Bsmt_SF: integer (nullable = true)
-- Heating: string (nullable = true)
-- Heating_QC: string (nullable = true)
-- Central_Air: string (nullable = true)
-- Electrical: string (nullable = true)
-- 1st_Flr_SF: integer (nullable = true)
-- 2nd_Flr_SF: integer (nullable = true)
-- Low_Qual_Fin_SF: integer (nullable = true)
-- Gr_Liv_Area: integer (nullable = true)
-- Bsmt_Full_Bath: integer (nullable = true)
-- Bsmt_Half_Bath: integer (nullable = true)
-- Full_Bath: integer (nullable = true)
-- Half_Bath: integer (nullable = true)
-- Bedroom_AbvGr: integer (nullable = true)
-- Kitchen_AbvGr: integer (nullable = true)
-- Kitchen_Qual: string (nullable = true)
-- TotRms_AbvGrd: integer (nullable = true)
-- Functional: string (nullable = true)
-- Fireplaces: integer (nullable = true)
-- Fireplace_Qu: string (nullable = true)
-- Garage_Type: string (nullable = true)
-- Garage_Yr_Blt: integer (nullable = true)
-- Garage_Finish: string (nullable = true)
-- Garage_Cars: integer (nullable = true)
-- Garage_Area: integer (nullable = true)
-- Garage_Qual: string (nullable = true)
-- Garage_Cond: string (nullable = true)
-- Paved_Drive: string (nullable = true)
-- Wood_Deck_SF: integer (nullable = true)
-- Open_Porch_SF: integer (nullable = true)
-- Enclosed_Porch: integer (nullable = true)
-- 3Ssn_Porch: integer (nullable = true)
-- Screen_Porch: integer (nullable = true)
-- Pool_Area: integer (nullable = true)
-- Pool_QC: string (nullable = true)
-- Fence: string (nullable = true)
-- Misc_Feature: string (nullable = true)
-- Misc_Val: integer (nullable = true)
-- Mo_Sold: integer (nullable = true)
-- Yr_Sold: integer (nullable = true)
-- Sale_Type: string (nullable = true)
-- Sale_Condition: string (nullable = true)
-- SalePrice: integer (nullable = true)

In [0]:
display(ames_df.describe())

summary,Order,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Year_Built,Year_Remod/Add,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Heating,Heating_QC,Central_Air,Electrical,1st_Flr_SF,2nd_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Functional,Fireplaces,Fireplace_Qu,Garage_Type,Garage_Yr_Blt,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,3Ssn_Porch,Screen_Porch,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Yr_Sold,Sale_Type,Sale_Condition,SalePrice
count,2930,2930,2930,2930,2440,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2907,2907,2930,2930,2930,2929,2929,2926,2929,2929,2928,2929,2929,2929,2930,2930,2930,2929,2930,2930,2930,2930,2928,2928,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2771,2928,2929,2929,2929,2929,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930
mean,1465.5,7.144644969887372E8,57.38737201365188,null,69.22459016393442,10147.921843003413,null,null,null,null,null,null,null,null,null,null,null,null,6.0948805460750854,5.563139931740614,1971.3563139931741,1984.266552901024,null,null,null,null,null,101.8968008255934,null,null,null,null,null,null,null,442.6295664049164,null,49.72243086377603,559.2625469443497,1051.6145442130419,null,null,null,null,1159.5576791808874,335.45597269624574,4.67679180887372,1499.6904436860068,0.43135245901639346,0.061133879781420764,1.5665529010238908,0.3795221843003413,2.8542662116040955,1.0443686006825939,null,6.443003412969284,null,0.5993174061433447,null,null,1978.1324431613136,null,1.7668146124957322,472.8197336975077,null,null,null,93.75187713310581,47.53344709897611,23.01160409556314,2.592491467576792,16.002047781569967,2.243344709897611,null,null,null,50.63515358361775,6.216040955631399,2007.790443686007,null,null,180796.0600682594
stddev,845.9624696167082,1.8873084464939E8,42.638024553802154,null,23.36533497450341,7880.017759439098,null,null,null,null,null,null,null,null,null,null,null,null,1.4110260835519741,1.1115365600303277,30.245360629374822,20.860285876849204,null,null,null,null,null,179.11261057727663,null,null,null,null,null,null,null,455.5908390911524,null,169.1684755915819,439.49415280392384,440.6150669617973,null,null,null,null,391.8908852534923,428.3957150088263,46.310510034470354,505.5088874720414,0.5248201879465199,0.2452535662517101,0.5529406116455404,0.5026292533151647,0.8277311419853722,0.21407624439170878,null,1.5729643963344617,null,0.6479209165512175,null,null,25.528411250924385,null,0.7605663649343566,215.04654854103072,null,null,null,126.36156187890569,67.48340013677372,64.13905920864136,25.14133103186944,56.08737022943252,35.59718061509771,null,null,null,566.3442882590897,2.7144924254301683,1.3166129226075387,null,null,79886.692356665
min,1,526301100,20,A (agr),21,1300,Grvl,Grvl,IR1,Bnk,AllPub,Corner,Gtl,Blmngtn,Artery,Artery,1Fam,1.5Fin,1,1,1872,1950,Flat,ClyTile,AsbShng,AsbShng,BrkCmn,0,Ex,Ex,BrkTil,Ex,Ex,Av,ALQ,0,ALQ,0,0,0,Floor,Ex,N,FuseA,334,0,0,334,0,0,0,0,0,0,Ex,2,Maj1,0,Ex,2Types,1895,Fin,0,0,Ex,Ex,N,0,0,0,0,0,0,Ex,GdPrv,Elev,0,1,2006,COD,Abnorml,12789
max,2930,1007100110,190,RM,313,215245,Pave,Pave,Reg,Lvl,NoSewr,Inside,Sev,Veenker,RRNn,RRNn,TwnhsE,SLvl,10,9,2010,2010,Shed,WdShngl,WdShing,Wd Shng,Stone,1600,TA,TA,Wood,TA,TA,No,Unf,5644,Unf,1526,2336,6110,Wall,TA,Y,SBrkr,5095,2065,1064,5642,3,2,4,2,8,3,TA,15,Typ,4,TA,NA,2207,Unf,5,1488,TA,TA,Y,1424,742,1012,508,576,800,TA,NA,TenC,17000,12,2010,WD,Partial,755000


In [0]:
display(ames_df.select("SalePrice"))

SalePrice
215000
105000
172000
244000
189900
195500
213500
191500
236500
189000


In [0]:
def display_missing_values(df):
  missing_df = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
  mask = missing_df.select([col(c) > 0 for c in missing_df.columns]).collect()[0].asDict().values()
  display(missing_df.select([column for column, flag in zip(df.columns, mask) if flag]))

  
def columns_with_missing_values_names(df):
  missing_df = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
  mask = missing_df.select([col(c) > 0 for c in missing_df.columns]).collect()[0].asDict().values()
  return [column for column, flag in zip(df.columns, mask) if flag]

display_missing_values(ames_df)

Lot_Frontage,Mas_Vnr_Type,Mas_Vnr_Area,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Electrical,Bsmt_Full_Bath,Bsmt_Half_Bath,Garage_Yr_Blt,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond
490,23,23,1,1,4,1,1,2,1,1,1,1,2,2,159,2,1,1,1,1


In [0]:
ames_doubles_df = ames_df
columns_with_missing = columns_with_missing_values_names(ames_doubles_df)
numeric_columns_to_impute = [field.name for field in ames_doubles_df.select(columns_with_missing).schema.fields if field.dataType == IntegerType()]
for c in numeric_columns_to_impute:
  ames_doubles_df = ames_doubles_df.withColumn(c, col(c).cast("double"))
  
ames_doubles_df.printSchema()

root
-- Order: integer (nullable = true)
-- PID: integer (nullable = true)
-- MS_SubClass: integer (nullable = true)
-- MS_Zoning: string (nullable = true)
-- Lot_Frontage: double (nullable = true)
-- Lot_Area: integer (nullable = true)
-- Street: string (nullable = true)
-- Alley: string (nullable = true)
-- Lot_Shape: string (nullable = true)
-- Land_Contour: string (nullable = true)
-- Utilities: string (nullable = true)
-- Lot_Config: string (nullable = true)
-- Land_Slope: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Condition_1: string (nullable = true)
-- Condition_2: string (nullable = true)
-- Bldg_Type: string (nullable = true)
-- House_Style: string (nullable = true)
-- Overall_Qual: integer (nullable = true)
-- Overall_Cond: integer (nullable = true)
-- Year_Built: integer (nullable = true)
-- Year_Remod/Add: integer (nullable = true)
-- Roof_Style: string (nullable = true)
-- Roof_Matl: string (nullable = true)
-- Exterior_1st: string (nullable = true)
-- Exterior_2nd: string (nullable = true)
-- Mas_Vnr_Type: string (nullable = true)
-- Mas_Vnr_Area: double (nullable = true)
-- Exter_Qual: string (nullable = true)
-- Exter_Cond: string (nullable = true)
-- Foundation: string (nullable = true)
-- Bsmt_Qual: string (nullable = true)
-- Bsmt_Cond: string (nullable = true)
-- Bsmt_Exposure: string (nullable = true)
-- BsmtFin_Type_1: string (nullable = true)
-- BsmtFin_SF_1: double (nullable = true)
-- BsmtFin_Type_2: string (nullable = true)
-- BsmtFin_SF_2: double (nullable = true)
-- Bsmt_Unf_SF: double (nullable = true)
-- Total_Bsmt_SF: double (nullable = true)
-- Heating: string (nullable = true)
-- Heating_QC: string (nullable = true)
-- Central_Air: string (nullable = true)
-- Electrical: string (nullable = true)
-- 1st_Flr_SF: integer (nullable = true)
-- 2nd_Flr_SF: integer (nullable = true)
-- Low_Qual_Fin_SF: integer (nullable = true)
-- Gr_Liv_Area: integer (nullable = true)
-- Bsmt_Full_Bath: double (nullable = true)
-- Bsmt_Half_Bath: double (nullable = true)
-- Full_Bath: integer (nullable = true)
-- Half_Bath: integer (nullable = true)
-- Bedroom_AbvGr: integer (nullable = true)
-- Kitchen_AbvGr: integer (nullable = true)
-- Kitchen_Qual: string (nullable = true)
-- TotRms_AbvGrd: integer (nullable = true)
-- Functional: string (nullable = true)
-- Fireplaces: integer (nullable = true)
-- Fireplace_Qu: string (nullable = true)
-- Garage_Type: string (nullable = true)
-- Garage_Yr_Blt: double (nullable = true)
-- Garage_Finish: string (nullable = true)
-- Garage_Cars: double (nullable = true)
-- Garage_Area: double (nullable = true)
-- Garage_Qual: string (nullable = true)
-- Garage_Cond: string (nullable = true)
-- Paved_Drive: string (nullable = true)
-- Wood_Deck_SF: integer (nullable = true)
-- Open_Porch_SF: integer (nullable = true)
-- Enclosed_Porch: integer (nullable = true)
-- 3Ssn_Porch: integer (nullable = true)
-- Screen_Porch: integer (nullable = true)
-- Pool_Area: integer (nullable = true)
-- Pool_QC: string (nullable = true)
-- Fence: string (nullable = true)
-- Misc_Feature: string (nullable = true)
-- Misc_Val: integer (nullable = true)
-- Mo_Sold: integer (nullable = true)
-- Yr_Sold: integer (nullable = true)
-- Sale_Type: string (nullable = true)
-- Sale_Condition: string (nullable = true)
-- SalePrice: integer (nullable = true)

In [0]:
for column in numeric_columns_to_impute:
  ames_doubles_df = ames_doubles_df.withColumn(column + "_NA", when(isnan(column) | col(column).isNull(), 1.0).otherwise(0.0))
  
ames_doubles_df.printSchema()

root
-- Order: integer (nullable = true)
-- PID: integer (nullable = true)
-- MS_SubClass: integer (nullable = true)
-- MS_Zoning: string (nullable = true)
-- Lot_Frontage: double (nullable = true)
-- Lot_Area: integer (nullable = true)
-- Street: string (nullable = true)
-- Alley: string (nullable = true)
-- Lot_Shape: string (nullable = true)
-- Land_Contour: string (nullable = true)
-- Utilities: string (nullable = true)
-- Lot_Config: string (nullable = true)
-- Land_Slope: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Condition_1: string (nullable = true)
-- Condition_2: string (nullable = true)
-- Bldg_Type: string (nullable = true)
-- House_Style: string (nullable = true)
-- Overall_Qual: integer (nullable = true)
-- Overall_Cond: integer (nullable = true)
-- Year_Built: integer (nullable = true)
-- Year_Remod/Add: integer (nullable = true)
-- Roof_Style: string (nullable = true)
-- Roof_Matl: string (nullable = true)
-- Exterior_1st: string (nullable = true)
-- Exterior_2nd: string (nullable = true)
-- Mas_Vnr_Type: string (nullable = true)
-- Mas_Vnr_Area: double (nullable = true)
-- Exter_Qual: string (nullable = true)
-- Exter_Cond: string (nullable = true)
-- Foundation: string (nullable = true)
-- Bsmt_Qual: string (nullable = true)
-- Bsmt_Cond: string (nullable = true)
-- Bsmt_Exposure: string (nullable = true)
-- BsmtFin_Type_1: string (nullable = true)
-- BsmtFin_SF_1: double (nullable = true)
-- BsmtFin_Type_2: string (nullable = true)
-- BsmtFin_SF_2: double (nullable = true)
-- Bsmt_Unf_SF: double (nullable = true)
-- Total_Bsmt_SF: double (nullable = true)
-- Heating: string (nullable = true)
-- Heating_QC: string (nullable = true)
-- Central_Air: string (nullable = true)
-- Electrical: string (nullable = true)
-- 1st_Flr_SF: integer (nullable = true)
-- 2nd_Flr_SF: integer (nullable = true)
-- Low_Qual_Fin_SF: integer (nullable = true)
-- Gr_Liv_Area: integer (nullable = true)
-- Bsmt_Full_Bath: double (nullable = true)
-- Bsmt_Half_Bath: double (nullable = true)
-- Full_Bath: integer (nullable = true)
-- Half_Bath: integer (nullable = true)
-- Bedroom_AbvGr: integer (nullable = true)
-- Kitchen_AbvGr: integer (nullable = true)
-- Kitchen_Qual: string (nullable = true)
-- TotRms_AbvGrd: integer (nullable = true)
-- Functional: string (nullable = true)
-- Fireplaces: integer (nullable = true)
-- Fireplace_Qu: string (nullable = true)
-- Garage_Type: string (nullable = true)
-- Garage_Yr_Blt: double (nullable = true)
-- Garage_Finish: string (nullable = true)
-- Garage_Cars: double (nullable = true)
-- Garage_Area: double (nullable = true)
-- Garage_Qual: string (nullable = true)
-- Garage_Cond: string (nullable = true)
-- Paved_Drive: string (nullable = true)
-- Wood_Deck_SF: integer (nullable = true)
-- Open_Porch_SF: integer (nullable = true)
-- Enclosed_Porch: integer (nullable = true)
-- 3Ssn_Porch: integer (nullable = true)
-- Screen_Porch: integer (nullable = true)
-- Pool_Area: integer (nullable = true)
-- Pool_QC: string (nullable = true)
-- Fence: string (nullable = true)
-- Misc_Feature: string (nullable = true)
-- Misc_Val: integer (nullable = true)
-- Mo_Sold: integer (nullable = true)
-- Yr_Sold: integer (nullable = true)
-- Sale_Type: string (nullable = true)
-- Sale_Condition: string (nullable = true)
-- SalePrice: integer (nullable = true)
-- Lot_Frontage_NA: double (nullable = false)
-- Mas_Vnr_Area_NA: double (nullable = false)
-- BsmtFin_SF_1_NA: double (nullable = false)
-- BsmtFin_SF_2_NA: double (nullable = false)
-- Bsmt_Unf_SF_NA: double (nullable = false)
-- Total_Bsmt_SF_NA: double (nullable = false)
-- Bsmt_Full_Bath_NA: double (nullable = false)
-- Bsmt_Half_Bath_NA: double (nullable = false)
-- Garage_Yr_Blt_NA: double (nullable = false)
-- Garage_Cars_NA: double (nullable = false)
-- Garage_Area_NA: double (nullable = false)

In [0]:
train_df, test_df = ames_doubles_df.randomSplit([0.8, 0.2], seed=1)

In [0]:
imputer = Imputer(strategy="median", inputCols=numeric_columns_to_impute, outputCols=numeric_columns_to_impute)
model = imputer.fit(train_df)
imputed_train_df = model.transform(train_df)
imputed_test_df = model.transform(test_df) 

In [0]:
cleaned_train_df = imputed_train_df.na.drop()
cleaned_test_df = imputed_test_df.na.drop()

In [0]:
display_missing_values(cleaned_train_df)

In [0]:
train_path = project_path + "train"
test_path = project_path + "test"

cleaned_train_df.write.format("delta").mode("overwrite").save(train_path)
cleaned_test_df.write.format("delta").mode("overwrite").save(test_path)